In [98]:
pip install yfinance


In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import timedelta

In [100]:
# List of stock tickers
tickers = [
    'ORC',   # Orchid Island Capital Inc
    'ARR',   # ARMOUR Residential REIT Inc
    'EARN',  # Ellington Residential Mortgage REIT
    'DX',    # Dynex Capital Inc
    'AGNC',  # AGNC Investment Corp
    'HRZN',  # Horizon Technology Finance Corp
    'PSEC',  # Prospect Capital Corp
    'PFLT',  # PennantPark Floating Rate Capital Ltd
    'EFC',   # Ellington Financial Inc
    'SCM',   # Stellus Capital Investment Corp
    'GOOD',  # Gladstone Commercial Corp
    'TWO',   # Two Harbors Investment Corp
    'SUNS',  # SLR Senior Investment Corp (formerly SUNS)
    'MITT',  # AG Mortgage Investment Trust Inc
    'INSW',  # International Seaways Inc
    'BKE',   # Buckle Inc
    'RGP',   # Resources Connection Inc
    'PANL',  # Pangaea Logistics Solutions Ltd
    'BEN',   # Franklin Resources Inc
    'TROW'   # T. Rowe Price Group Inc
]

start_date = "2015-05-01"
end_date = "2025-05-01"

In [101]:
dividends = {}
prices = {}

In [102]:
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        div = stock.dividends
        # Filter for the last 10 years
        div = div[(div.index >= start_date) & (div.index <= end_date)]
        if not div.empty:
            df = div.reset_index()
            df.columns = ['Date', 'Dividend']
            df['Ticker'] = ticker
            df.to_csv(f"{ticker}_dividends_10y.csv", index=False)
            dividends[f"{ticker}_div"] = df.copy(deep=True)
            print(f"[✓] Saved 10-year dividend data for {ticker}")
        else:
            print(f"[!] No dividend data found for {ticker}")
    except Exception as e:
        print(f"[X] Failed to fetch data for {ticker}: {e}")



[✓] Saved 10-year dividend data for ORC
[✓] Saved 10-year dividend data for ARR
[✓] Saved 10-year dividend data for EARN
[✓] Saved 10-year dividend data for DX
[✓] Saved 10-year dividend data for AGNC
[✓] Saved 10-year dividend data for HRZN
[✓] Saved 10-year dividend data for PSEC
[✓] Saved 10-year dividend data for PFLT
[✓] Saved 10-year dividend data for EFC
[✓] Saved 10-year dividend data for SCM
[✓] Saved 10-year dividend data for GOOD
[✓] Saved 10-year dividend data for TWO
[!] No dividend data found for SUNS
[✓] Saved 10-year dividend data for MITT
[✓] Saved 10-year dividend data for INSW
[✓] Saved 10-year dividend data for BKE
[✓] Saved 10-year dividend data for RGP
[✓] Saved 10-year dividend data for PANL
[✓] Saved 10-year dividend data for BEN
[✓] Saved 10-year dividend data for TROW


In [103]:
for ticker in tickers:
  try:
    price = yf.download(ticker, start=start_date, end=end_date, progress=False)
    if not price.empty:
        price.reset_index(inplace=True)
        price['Ticker'] = ticker
        price.columns = [col[0] if isinstance(col, tuple) else col for col in price.columns]
        price.to_csv(f"{ticker}_prices_10y.csv", index=False)
        prices[f"{ticker}_price"] = price.copy(deep=True)
        print(f"[✓] Saved EOD data for {ticker}")
    else:
        print(f"[!] No EOD price data for {ticker}")

  except Exception as e:
    print(f"[X] Error for {ticker}: {e}")

[✓] Saved EOD data for ORC
[✓] Saved EOD data for ARR
[✓] Saved EOD data for EARN
[✓] Saved EOD data for DX
[✓] Saved EOD data for AGNC
[✓] Saved EOD data for HRZN
[✓] Saved EOD data for PSEC
[✓] Saved EOD data for PFLT
[✓] Saved EOD data for EFC
[✓] Saved EOD data for SCM
[✓] Saved EOD data for GOOD
[✓] Saved EOD data for TWO
[✓] Saved EOD data for SUNS
[✓] Saved EOD data for MITT
[✓] Saved EOD data for INSW
[✓] Saved EOD data for BKE
[✓] Saved EOD data for RGP
[✓] Saved EOD data for PANL
[✓] Saved EOD data for BEN
[✓] Saved EOD data for TROW


In [104]:
filtered_data = {}

In [105]:
for div_key in dividends:
  base_ticker = div_key.replace("_div", "")
  price_key = f"{base_ticker}_price"
  if price_key not in prices:
    continue
  div_df = dividends[div_key].copy(deep=True)
  price_df = prices[price_key].copy(deep=True)

  div_df['Date'] = pd.to_datetime(div_df['Date']).dt.date
  price_df['Date'] = pd.to_datetime(price_df['Date']).dt.date

  ex_dates = set(div_df['Date'])
  ex_date_prev = (t - timedelta(days=1) for t in ex_dates)
  relevant_dates = ex_dates.union(ex_date_prev)

  filtered_price = price_df[price_df['Date'].isin(relevant_dates)].copy(deep=True)
  filtered_price['tag'] = filtered_price['Date'].apply(lambda d: 'Ex' if d in ex_dates else 'Ex-1')

  merged = pd.merge(filtered_price, div_df[['Date', 'Dividend']], on='Date', how='left')
  filtered_data[base_ticker] = merged.copy(deep=True)

In [106]:
def calc_return_rate(profit, initial):
  return profit/initial

In [107]:
def calc_win_rate(profit):
  return np.sign(profit)

In [108]:
def calculate_metrics(group, name):
  V0 = group['Open']
  diff= group['Close'] - group['Open']
  if name == 'Ex':
    diff = -diff
  return_rates = calc_return_rate(diff, V0)
  wins = calc_win_rate(diff)
  group = group.copy()
  group['return'] = return_rates
  group['wins'] = wins
  return group

In [109]:
for ticker in filtered_data:
  returns = (
      filtered_data[ticker]
      .groupby('tag', group_keys=False)
      .apply(lambda grp: calculate_metrics(grp, grp.name), include_groups=False)
  )
  filtered_data[ticker]['returns'] = 1+returns['return']
  filtered_data[ticker]['wins'] = returns['wins']


In [110]:
results = pd.DataFrame()

In [111]:
stats = {
    ticker : {
        "win_rate": df['wins'].eq(1).sum()/len(df['wins']),
        "loss_rate": df['wins'].eq(-1).sum()/len(df['wins']),
        'avg_raw_return': (df['returns'] - 1).mean(),
        'simple_return_sum':  (df['returns'] - 1).sum(),
        "compunded_return": df['returns'].prod() - 1
    }
    for ticker, df in filtered_data.items()
}

In [112]:
results = pd.DataFrame.from_dict(stats, orient='index')

In [113]:
results

,win_rate,loss_rate,avg_raw_return,simple_return_sum,compunded_return
ORC,0.518349,0.412844,0.003158,0.688389,0.897616
ARR,0.562212,0.410138,0.001643,0.356586,0.370807
EARN,0.496000,0.440000,0.001102,0.137793,0.127995
DX,0.494048,0.458333,0.001843,0.309554,0.331045
AGNC,0.486364,0.463636,0.000131,0.028759,0.007239
HRZN,0.620690,0.349754,0.005178,1.051161,1.751011
PSEC,0.522727,0.422727,0.002105,0.463064,0.558104
PFLT,0.522727,0.418182,0.002088,0.459328,0.532675
EFC,0.445122,0.548780,-0.000294,-0.048204,-0.073799
SCM,0.528846,0.423077,0.002628,0.546636,0.689944


In [114]:
results.describe()

,win_rate,loss_rate,avg_raw_return,simple_return_sum,compunded_return
count,19.000000,19.000000,19.000000,19.000000,19.000000
mean,0.494427,0.468944,0.000405,0.163194,0.220865
std,0.056905,0.062752,0.002403,0.352178,0.500182
min,0.375000,0.349754,-0.006093,-0.299446,-0.346374
25%,0.459993,0.422902,-0.000773,-0.069816,-0.086342
50%,0.496000,0.463636,0.000131,0.026931,0.007239
75%,0.525787,0.496668,0.001965,0.407957,0.451741
max,0.620690,0.600000,0.005178,1.051161,1.751011
